# Simulating the Harber Bosch process with quantum computing

## Progress

- [x] Hamiltonian `H_1` for all components
- [ ] Reduce number of terms for `H_1`
- [ ] LCU to simulate `H_1`

## Code

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

### Define Hamiltonian

In [3]:
symbols = ["Fe", "N", "N", "H", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 
                        0.0, 0.0, 0.2,
                        0.0, 0.0, 0.3,
                        0.0, 0.0, -0.2,
                        0.0, 0.0, -0.3,
                       ])
# H, qubits = qchem.molecular_hamiltonian(symbols, coordinates, method='pyscf')

In [5]:
qubits

60

In [24]:
dataset_N2 = qml.data.load("qchem", molname="N2", bondlength=1.12, basis="STO-3G")
dataset_N2[0]

<Dataset = molname: N2, basis: STO-3G, bondlength: 1.12, attributes: ['basis', 'basis_rot_groupings', ...]>

In [29]:
H_N2 = dataset_N2[0].hamiltonian

In [30]:
H_N2

<Hamiltonian: terms=2951, wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]>


For iron, the outer orbitals are $4s2, 3d6$. They are configured as
- $4s2:\uparrow \downarrow$
- $3d6:\uparrow \downarrow,\uparrow,\uparrow,\uparrow,\uparrow $

Therefore there are four active oribitals

In [17]:
symbols_Fe = ["Fe"]
coordinates_Fe = np.array([0.0, 0.0, 0.0])
H_Fe, qubits_Fe = qchem.molecular_hamiltonian(symbols_Fe, coordinates_Fe, 
                                              active_orbitals=14, method='pyscf')
# 4 doesn't work somehow
# with setting 14 active orbitals: 23k terms
# without setting: 66k terms

In [16]:
H_Fe

<Hamiltonian: terms=23319, wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]>


### Simulate Hamiltonian

In [38]:
qml.matrix(H_Fe)


KeyboardInterrupt



## Next step / meeting minute

### 2nd week

### 1st week
1. Use params `active_electrons`, `active_orbitals` inside `molecular_hamiltonian`
    1. Principled way to do it without knowing Fe is catalyst?
       > valence electron in the outer shell, choose the radius (coordinate?)
    2. Coordinates
       > coordinate for reactant, coordinate for products may be in somewhere else
       > evaluate energy at a lot of different coordinates?
       > Put H2 N2 Fe closer and then farther to see what happens. There must be a method
    3. QPE
       > Time evolution. Trotter, LCU (build different matrix, one of them is time evolution), Qubitizations
2. ZNE?
    1. If it concat a chain of gates, could the real system reach the end of the gates?
    2. Doesn't seem to need more qubit?